# Algorytm Little'a dla problemu TSP

Dominik Matracki 408558 <br>
Patryk Lyczko

In [7]:
import numpy as np

In [16]:

def reduce_matrix(M):
    """Reduces rows and columns"""
    result = M.copy()

    cost = 0
    for row in result:
        minimal = min(row)
        # Jezeli najmniejsza wartoscia jest inf to omijamy
        if minimal != np.inf:
            row -= minimal
            cost += minimal

    minimal = np.min(result, axis=0)

    for i, value in enumerate(minimal):
        if value != np.inf:
            result[:,i] -= value
            cost += value

    return result, cost

def find_zeros(M):
    """Find all zeros in matrix"""
    n, _ = M.shape
    zeros = []
    for i in range(n):
        for j in range(n):
            if M[i, j] == 0:
                zeros.append((i,j))
    return zeros

def optimistic_cost(M, zeros):
    n, _ = M.shape
    minimal = np.inf
    position = zeros[0]

    for row, col in zeros:
        minimalCol = np.inf
        minimalRow = np.inf
        for i in range(n):
            if col != i and M[row, i] < minimalCol:
                minimalCol = M[row, i]
            
            if row != i and M[i, col] < minimalRow:
                minimalRow = M[i, col]

        if minimalCol + minimalRow < minimal:
            minimal = minimalCol + minimalRow
            position = row, col

    return minimal, position

def block_cycles(M, trip):
    n, _ = M.shape
    if len(trip) == n - 1:
        return

    addedEdge = trip[-1]
    row, col = addedEdge
    beginings = [x for x, _ in trip]
    endings = [y for _, y in trip]

    for begin in beginings:
        M[col,begin] = np.inf
    
    for end in endings:
        M[end,row] = np.inf

class SubProblem:
    def __init__(self, matrix, trip, lower_bound):
        self.matrix = matrix
        self.lower_bound = lower_bound
        self.trip = trip

class PriorityQueue:
    """Kolejka priorytetowa na szybko"""
    def __init__(self):
        self.queue = []

    def enqueue(self, problem):
        self.queue.append(problem)
        self.queue.sort(key=lambda problem: problem.lower_bound)

    def dequeue(self):
        return self.queue.pop(0)
    
    def is_empty(self):
        return len(self.queue) == 0

def tsp_little(M):
    n, _ = M.shape

    # Redukcja macierzy i wyznaczenie lower bound
    matrix, lb = reduce_matrix(M)

    # Aktualny problem
    problem = SubProblem(matrix=matrix, trip=[], lower_bound=lb)
    priority_queue = PriorityQueue()
    run = True

    # Inicjalizacja najlepszego rozwiazania
    bestSolution = {
        "trip": [],
        "lower_bound": np.inf,
    }

    while run:
        # Wyznaczenie zer w macierzy
        zeros = find_zeros(problem.matrix)

        # Brak znalezionych zer, znaleziono jakies rozwiazanie
        if len(zeros) == 0:

            # Sprawdzamy czy rozwiazanie jest rozwiazaniem dopuszczalnym
            if len(problem.trip) == n:
                # Jezeli tak to sprawdzamy czy jest lepsze od wartosci odcinajacej
                if bestSolution["lower_bound"] > problem.lower_bound:
                    # Jezeli tak to zapisujemy rozwiazanie najlepsze
                    bestSolution["lower_bound"] = problem.lower_bound
                    bestSolution["trip"] = problem.trip

            # Sprawdzamy czy mamy jeszcze podproblemy do rozwiazania
            if priority_queue.is_empty():
                return bestSolution

            problem = priority_queue.dequeue()
            continue

        # Wyznaczamy najlepsza pozycje
        _, position = optimistic_cost(problem.matrix, zeros)

        p1 = problem.matrix.copy()
        p2 = problem.matrix.copy()
        row, col = position

        # Podproblem 1 - dodajemy rozwiazanie

        # Wykreslamy wiersz i kolumne
        p1[row,:] = np.inf
        p1[:,col] = np.inf

        # Zabraniamy podcykli
        block_cycles(p1, problem.trip+[position])

        # Wyznaczamy nowe lower bound
        matrix1, lb1 = reduce_matrix(p1)

        # Podproblem 2 - zabraniamy rozwiazania
        p2[row,col] = np.inf
        matrix2, lb2 = reduce_matrix(p2)

        # Wybieramy podproblem o mniejszym lower bound
        if lb1 < lb2:
            problem.matrix = matrix1
            problem.lower_bound += lb1
            problem.trip.append((row, col))

            # Dodajemy drugi subproblem do kolejki
            if bestSolution["lower_bound"] > lb + lb2:
                priority_queue.enqueue(SubProblem(matrix2, problem.trip[:-1], lb+lb2))
            continue

        problem.matrix = matrix2
        problem.lower_bound += lb2

        # Dodajemy drugi subproblem do kolejki
        if bestSolution["lower_bound"] > lb + lb2:
            priority_queue.enqueue(SubProblem(matrix1, problem.trip+[(row, col)], lb+lb1))
    


In [17]:
M = np.array([
    [np.inf, 4, 8, 7],
    [8, np.inf, 7, 2],
    [5, 2, np.inf, 1],
    [7, 1, 2, np.inf],
])
tsp_little(M)

{'trip': [(1, 0), (0, 3), (2, 1), (3, 2)], 'lower_bound': 13.0}